In [1]:
from tqdm import tqdm
import numpy as np
import os, glob
import shutil
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.nn import functional as F

import sys
sys.path.append('/raid-dgx1/Hasnat/Covoiturage/')
from my_utilities_display_related import show_image, show_image_from_file, show_image_subplots

os.environ["CUDA_VISIBLE_DEVICES"]="1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print((torch.cuda.is_available(), torch.version.cuda)) 

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

(True, '10.1')


In [2]:
from dataset import Yolo_dataset
from tool.tv_reference.utils import collate_fn as val_collate

from cfg import Cfg
import argparse
from easydict import EasyDict as edict

def get_args(**kwargs):
    cfg = kwargs
    parser = argparse.ArgumentParser(description='Train the Model on images and target masks',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('-b', '--batch-size', metavar='B', type=int, nargs='?', default=2,
    #                     help='Batch size', dest='batchsize')
    parser.add_argument('-l', '--learning-rate', metavar='LR', type=float, nargs='?', default=0.001,
                        help='Learning rate', dest='learning_rate')
    parser.add_argument('-f', '--load', dest='load', type=str, default=None,
                        help='Load model from a .pth file')
    parser.add_argument('-g', '--gpu', metavar='G', type=str, default='-1',
                        help='GPU', dest='gpu')
    parser.add_argument('-dir', '--data-dir', type=str, default=None,
                        help='dataset dir', dest='dataset_dir')
    parser.add_argument('-pretrained', type=str, default=None, help='pretrained yolov4.conv.137')
    parser.add_argument('-classes', type=int, default=80, help='dataset classes')
    parser.add_argument('-train_label_path', dest='train_label', type=str, default='train.txt', help="train label path")
    parser.add_argument(
        '-optimizer', type=str, default='adam',
        help='training optimizer',
        dest='TRAIN_OPTIMIZER')
    parser.add_argument(
        '-iou-type', type=str, default='iou',
        help='iou type (iou, giou, diou, ciou)',
        dest='iou_type')
    parser.add_argument(
        '-keep-checkpoint-max', type=int, default=10,
        help='maximum number of checkpoints to keep. If set 0, all checkpoints will be kept',
        dest='keep_checkpoint_max')
    args = vars(parser.parse_args())

    # for k in args.keys():
    #     cfg[k] = args.get(k)
    cfg.update(args)

    return edict(cfg)

def collate(batch):
    images = []
    bboxes = []
    for img, box in batch:
        images.append([img])
        bboxes.append([box])
    images = np.concatenate(images, axis=0)
    images = images.transpose(0, 3, 1, 2)
    images = torch.from_numpy(images).div(255.0)
    bboxes = np.concatenate(bboxes, axis=0)
    bboxes = torch.from_numpy(bboxes)
    return images, bboxes

In [3]:
cfg = get_args(**Cfg)
cfg.dataset_dir = '/raid-dgx1/Hasnat/pytorch-YOLOv4/voc_images/'
config = cfg
cfg.batch = 16
train_dataset = Yolo_dataset(config.train_label, config, train=True)
val_dataset = Yolo_dataset(config.val_label, config, train=False)

n_train = len(train_dataset)
n_val = len(val_dataset)

print(cfg.dataset_dir)
train_loader = DataLoader(train_dataset, batch_size=config.batch // config.subdivisions, shuffle=True,
                          num_workers=16, pin_memory=False, drop_last=True, collate_fn=collate)

val_loader = DataLoader(val_dataset, batch_size=config.batch // config.subdivisions, shuffle=True, num_workers=8,
                            pin_memory=True, drop_last=True, collate_fn=val_collate)
print(cfg.dataset_dir)

/raid-dgx1/Hasnat/pytorch-YOLOv4/voc_images/
/raid-dgx1/Hasnat/pytorch-YOLOv4/voc_images/


In [7]:
print(cfg.dataset_dir)
ds = val_loader.dataset
for img_idx in range(len(ds)):
    img, targets = ds[img_idx]
    
    print(targets["image_id"].item())
    break
    #img, targets = ds[img_idx]

/raid-dgx1/Hasnat/pytorch-YOLOv4/voc_images/
89379579


In [5]:
from tool.tv_reference.coco_utils import convert_to_coco_api
coco = convert_to_coco_api(train_loader.dataset, bbox_fmt='coco')

in function convert_to_coco_api...
/raid-dgx1/Hasnat/pytorch-YOLOv4/voc_images/ 002361.jpg


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
#for i, batch in train_loader:
#    print(i, batch)
#    break

In [ ]:
#cfg

In [9]:
all_files = open('data/val.txt').readlines()

#
all_files[0]

'007903.jpg 99,142,499,374,17 360,102,500,254,14 0,72,184,374,14 \n'

In [34]:
import hashlib
all_int_id = []
for t_ in val_dataset.truth.keys():
    tiid = int(hashlib.sha1(t_.encode("utf-8")).hexdigest(), 16) % (10 ** 10)
    all_int_id.append(tiid)

In [35]:
len(all_int_id), len(val_dataset.truth.keys()), len(set(all_int_id))

(4952, 4952, 4952)